In [1]:
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf
from tensorboard.plugins import projector
import os
from tqdm import tqdm

log_dir = './logs/phi1_5/vocab/' # Tensorboard log directory

/Users/kagan/Desktop/Documents/Programmierung/XAI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/kagan/Desktop/Documents/Programmierung/XAI/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")
model = AutoModel.from_pretrained("microsoft/phi-1_5")

# Word embeddings
word_embeddings = model.embed_tokens.weight

In [3]:
# Import all vocabs from tokenizer
vocab_list = sorted(tokenizer.vocab.items(), key=lambda x:x[1])

# Create log directory if not exists
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save the embeddings as tsv file
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
    for word, idx in tqdm(vocab_list):
        f.write("{}\n".format(str(word.encode(encoding='iso-8859-1', errors='replace'))))

100%|██████████| 50295/50295 [00:00<00:00, 1077888.70it/s]


In [4]:
# Extract embeddings and convert to TensorFlow Variable
embeddings = tf.Variable(model.embed_tokens.weight.detach().numpy())
checkpoint = tf.train.Checkpoint(embedding=embeddings)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

'./logs/phi1_5/vocab/embedding.ckpt-1'

In [5]:
# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()

# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)